In [1]:
import torch
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch import optim
from torch import nn
from torch.utils.data import DataLoader
from tqdm import tqdm

We define our CNN module class which will define the layers of the CNN in the constructor and implement the forward method

In [2]:
class CNN(nn.Module):
    
    def __init__(self, in_channels, num_classes = 10):
        
        super().__init__()
        
        self.conv1 = nn.Conv2d(in_channels = in_channels, out_channels = 8, kernel_size = 3, stride = 1, padding = 1)
        
        self.pool = nn.MaxPool2d(kernel_size = 2, stride = 2)
        
        self.conv2 = nn.Conv2d(in_channels = 8, out_channels = 16, kernel_size = 3, stride = 1, padding = 1)
        
        self.fc1 = nn.Linear(16 * 7 *7, num_classes)
        
    def forward(self, x):
        
        x = F.relu(self.conv1(x))
        
        x = self.pool(x)
        
        x = F.relu(self.conv2(x))
        
        x = self.pool(x)
        
        x = x.reshape(x.shape[0], -1)
        
        x = self.fc1(x)
        
        return x
        

In [11]:
input_size = 784
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 10

In [12]:
import ssl
import certifi

# Fix for SSL certificate issue on macOS
ssl._create_default_https_context = lambda: ssl.create_default_context(cafile=certifi.where())

# Define batch size
batch_size = 64  # or any value you prefer

# Transform: convert images to tensors
transform = transforms.ToTensor()

# Load training dataset
train_dataset = datasets.MNIST(root="dataset/", train=True, download=True, transform=transform)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

# Load test dataset
test_dataset = datasets.MNIST(root="dataset/", train=False, download=True, transform=transform)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

train_dataset = datasets.MNIST(root = "dataset/", download = True, train = True, transform = transforms.ToTensor())
train_loader = DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle = True)

test_dataset = datasets.MNIST(root = "dataset/", download = True, train = False, transform = transforms.ToTensor())
test_loader = DataLoader(dataset = test_dataset, batch_size = batch_size, shuffle = True)

In [15]:
model = CNN(in_channels = 1, num_classes = num_classes)

In [16]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

In [21]:
for epoch in range(num_epochs):
    
    print(f'Epoch [{epoch +  1}/{num_epochs}]')
    
    for batch_index, (data, targets) in enumerate(tqdm(train_loader)):
        
        predictions = model(data)
        
        loss = criterion(predictions, targets)
        
        optimizer.zero_grad()
        
        loss.backward()
        
        optimizer.step()

Epoch [1/10]


100%|██████████| 938/938 [00:14<00:00, 63.44it/s]


Epoch [2/10]


100%|██████████| 938/938 [00:14<00:00, 63.57it/s]


Epoch [3/10]


...

100%|██████████| 938/938 [00:15<00:00, 61.37it/s]


Epoch [9/10]


100%|██████████| 938/938 [00:15<00:00, 61.67it/s]


Epoch [10/10]


100%|██████████| 938/938 [00:14<00:00, 62.76it/s]


In [22]:
def check_accuracy(loader, model):
    
    if loader.dataset.train:
        print('Checking accuracy on training data')
    else:
        print('Checking accuracy on test data')
        
    num_correct = 0
    num_samples = 0
    model.eval()
    
    with torch.no_grad():
        
        for x, y in loader:
            
            predictions = model(x)
            
            _, predictions = predictions.max(1)
            
            num_correct += (predictions == y).sum()
            
            num_samples += predictions.size(0)
            
        accuracy = float(num_correct) / float(num_samples) * 100
        
        print(f'Got {num_correct}/{num_samples} with accuracy {accuracy:.2f}%')
        
    model.train()
    
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking accuracy on training data
Got 59459/60000 with accuracy 99.10%
Checking accuracy on test data
Got 9858/10000 with accuracy 98.58%
